# Dimensionality reduction

---
Author: Anatoliy Durkin

Updated: 14.04.2025

---
В ноутбуке рассмотрены основные методы снижения размерности данных.

В реальных данных часто сотни или тысячи признаков (например, изображения, тексты, геномные данные). Проблемы:

- Проклятие размерности: При большом числе признаков данные становятся разреженными, модели хуже работают.
- Шум и избыточность: Многие признаки коррелируют или не несут полезной информации.
- Визуализация: Человек не может воспринимать >3 измерений.

Основные подходы
- Линейные методы: PCA, LDA.
- Нелинейные методы: t-SNE, UMAP.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler

# Линейные методы

## Метод главных компонент (PCA)

In [ ]:
peng = pd.read_csv('penguins.csv')

In [ ]:
peng = peng[peng['sex']!='.']
peng = peng[(peng['flipper_length_mm']<1000) & (peng['flipper_length_mm']>100)]
peng = peng.dropna().reset_index(drop=True)

In [ ]:
peng.head()

In [ ]:
sns.pairplot(peng, hue='sex')

In [ ]:
sns.heatmap(peng.corr(), annot=True, cmap='coolwarm', center=0)

In [ ]:
X_peng = peng.drop(['sex'], axis=1)
X_peng = StandardScaler().fit_transform(X_peng)
y_peng = pd.get_dummies(peng['sex'], drop_first=True)

In [ ]:
y_peng.head()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X_peng_pca = pca.fit_transform(X_peng)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
plt.scatter(X_peng_pca[:, 0], X_peng_pca[:, 1], c=y_peng.values)
plt.colorbar()
plt.xlabel("PC1 (Дисперсия: {:.2%})".format(pca.explained_variance_ratio_[0]))
plt.ylabel("PC2 (Дисперсия: {:.2%})".format(pca.explained_variance_ratio_[1]))

Выбрать оптимальное число компонент можно с использованием метода "локтя".

In [ ]:
pca_full = PCA().fit(X_peng)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.axhline(y=0.95, color='r', linestyle='--')  # 95% дисперсии
plt.xticks(range(X_peng.shape[1]), range(1, X_peng.shape[1]+1))
plt.xlabel('Компонент')
plt.ylabel('Объясненная дисперсия')
plt.grid()

In [ ]:
pca = PCA(n_components=3)
X_peng_pca = pca.fit_transform(X_peng)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=X_peng_pca[:, 0], y=X_peng_pca[:, 1], z=X_peng_pca[:, 2],
                                   mode='markers', marker=dict(color=y_peng.values))],
               layout=dict(height=700))
fig.show()

In [ ]:
cancer = pd.read_csv('breast_cancer.csv')

In [ ]:
cancer

Разделите данные на X и y, а затем воспользуйтесь методом локтя для определения оптимального количество компонент.

In [ ]:
# Ваш код
...

## Линейный дискриминантный анализ (LDA)

Отличие от PCA: Учитывает метки классов для максимизации межклассового расстояния.

Когда использовать: Когда классы хорошо разделимы.

In [ ]:
stud = pd.read_csv('Student_performance_data.csv', index_col=0)

In [ ]:
stud

In [ ]:
features = stud.drop(['GradeClass'], axis=1)
features = StandardScaler().fit_transform(features)
target = stud['GradeClass']

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(features, target)

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(features)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.scatter(X_pca[:, 0], X_pca[:, 1], c=target)
ax1.set_title("PCA")
ax2.scatter(X_lda[:, 0], X_lda[:, 1], c=target)
ax2.set_title("LDA")

# Нелинейные методы

## t-SNE

Идея: Сохраняет локальные расстояния между точками.

Ключевые параметры:
- `perplexity`: Баланс между локальной/глобальной структурой (обычно 5-50).
- `random_state`: Для воспроизводимости.

t-SNE требует нормализованных данных.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_peng)

In [ ]:
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_peng.values)
plt.colorbar()
plt.title("t-SNE: Penguins dataset (perplexity=30)")

## UMAP

Если библиотека отсутсвует, ее можно установить:

In [ ]:
# !pip install umap-learn

In [ ]:
import umap.umap_ as umap

Преимущества перед t-SNE:

- Быстрее (оптимизированный алгоритм).
- Лучше сохраняет глобальную структуру.

Ключевые параметры:
- `n_neighbors`: Аналог perplexity (обычно 5-50).
- `min_dist`: Минимальное расстояние между точками (0.1-0.5).

In [ ]:
umap_model = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42)
X_umap = umap_model.fit_transform(X_peng)

In [ ]:
plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y_peng.values)
plt.colorbar()
plt.title("UMAP: Penguins dataset")